In [1]:
file_id = '18Wx4PmuM3A4L3cpeEzM71WZf4HybVJ3o'  # замените на ваш идентификатор файла
file_name = 'hack.zip'  # замените на желаемое имя файла и расширение

!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id={file_id}' -O {file_name}

# https://drive.google.com/file/d//view?usp=sharing
file_id = '1bCI9KbpJCDURD3v8D92-oEWTgqOpiIh2'  # замените на ваш идентификатор файла
file_name = 'model.zip'  # замените на желаемое имя файла и расширение

!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id={file_id}' -O {file_name}


--2024-06-11 12:29:25--  https://docs.google.com/uc?export=download&id=18Wx4PmuM3A4L3cpeEzM71WZf4HybVJ3o
Resolving docs.google.com (docs.google.com)... 173.194.195.138, 173.194.195.113, 173.194.195.102, ...
Connecting to docs.google.com (docs.google.com)|173.194.195.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=18Wx4PmuM3A4L3cpeEzM71WZf4HybVJ3o&export=download [following]
--2024-06-11 12:29:25--  https://drive.usercontent.google.com/download?id=18Wx4PmuM3A4L3cpeEzM71WZf4HybVJ3o&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 108.177.121.132, 2607:f8b0:4001:c19::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|108.177.121.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 182842 (179K) [application/octet-stream]
Saving to: ‘hack.zip’

hack.zip            100%[===================>] 178.56K  --.-KB/s    in

In [2]:
!unzip hack.zip
!unzip model.zip

Archive:  hack.zip
  inflating: 9. ДЗМ/Количество исследований за месяц (для проверки).xlsx  
  inflating: 9. ДЗМ/Количество исследований по неделям (для реализации).xlsx  
  inflating: 9. ДЗМ/Нормативы РЦ.xlsx  
  inflating: 9. ДЗМ/Пример табеля с количеством врачей в разрезе модальностей и доп модальностей.xlsx  
  inflating: 9. ДЗМ/Шаблон для заполнения проверки по прогнозу за Февраль.xlsx  
Archive:  model.zip
   creating: model/
  inflating: model/last.pt           
  inflating: model/best.pt           


In [6]:
# prompt: создай пять dataframe по каждому файлу xlsx которые лежат в 9. ДЗМ

import pandas as pd
import os

# Создаем словарь для хранения датафреймов
df_dict = {}

# Перебираем файлы в папке 9. ДЗМ
for file_name in os.listdir('9. ДЗМ'):
  # Проверяем, что файл имеет расширение xlsx
  if file_name.endswith('.xlsx'):
    # Читаем файл xlsx и сохраняем его в словаре
    df_dict[file_name] = pd.read_excel(os.path.join('9. ДЗМ', file_name))




In [7]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np


class CustomDataset(Dataset):
    def __init__(self, dataframe, window_size, prev_weeks):
        self.data = dataframe
        self.window_size = window_size
        self.prev_weeks = prev_weeks

        # Удаление строк с NaN значениями (в конце)
        self.data.dropna(inplace=True)

        # Создание входных и выходных переменных
        self.X, self.y = self.create_sequences_old()

    def create_sequences_old(self):
        X, y = [], []
        for i in range(len(self.data) - self.window_size):
            X.append(self.data.iloc[i:i+self.window_size, :12].values)
            y.append(self.data.iloc[i+self.window_size, 2:12].values)
        return np.array(X), np.array(y)

    def create_sequences(self):
        X, y = [], []
        for i in range(52, len(self.data) - self.window_size - 2):
            print(i + self.window_size - 1 - (52 + self.prev_weeks // 2 + 1), i + self.window_size - 1 - (52 - self.prev_weeks // 2))
            _1 = self.data.iloc[i:i+self.window_size, :12].values
            _2 = self.data.iloc[i + self.window_size - (52 + self.prev_weeks // 2): i + self.window_size - (52 - self.prev_weeks // 2) , :12].values
            print(_2.shape)
            print(_1.shape)
            X.append(np.concatenate((_1, _2), axis=0))
            # _1 = self.data.iloc[i+self.window_size-1, :2]
            # _2 = self.data.iloc[i+self.window_size-1, 12:].values
            # y.append(np.concatenate((_1, _2), axis=0))
            y.append(self.data.iloc[i+self.window_size-1, 12:].values)
        return np.array(X), np.array(y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        X = torch.tensor(self.X[idx], dtype=torch.float32)
        y = torch.tensor(self.y[idx], dtype=torch.float32)
        return X, y
    def append(self,x, y):
      self.X = np.append(self.X, x, axis=0)
      self.y = np.append(self.y, y.detach().numpy(), axis = 0)

window_size = 5
prev_weeks = 5


NameError: name 'dataset' is not defined

In [30]:
import torch.nn as nn
import torch.optim as optim

def cosine_distance(tensor1, tensor2):
    """
    Вычисляет косинусное расстояние между двумя тензорами.

    Args:
    tensor1 (torch.Tensor): Первый тензор.
    tensor2 (torch.Tensor): Второй тензор.

    Returns:
    float: Косинусное расстояние между двумя тензорами.
    """
    cos = nn.CosineSimilarity(dim=0, eps=1e-6)
    return 1 - cos(tensor1, tensor2)


class CustomHuberLoss(nn.Module):
    def __init__(self, delta=1.0):
        super(CustomHuberLoss, self).__init__()
        self.delta = delta

    def forward(self, outputs, targets):
        huber_loss = torch.where(
            torch.abs(outputs - targets) < self.delta,
            0.5 * (outputs - targets) ** 2,
            self.delta * (torch.abs(outputs - targets) - 0.5 * self.delta)
        )
        penalty = torch.sum(torch.relu(-outputs))  # штраф за отрицательные значения
        return torch.mean(huber_loss) + penalty

# Определение модели нейронной сети
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

def custom_loss(outputs, targets):
    mse_loss = nn.MSELoss()(outputs, targets)
    penalty = torch.sum(torch.relu(-outputs)) * 2 # штраф за отрицательные значения
    return mse_loss + penalty


def predict_n_weeks(model, dataset, n_weeks):
    model.eval()
    predictions = []

    # Берем последние window_size недель для инициализации
    # last_x = dataset.X[-1].reshape(1, -1, 10)
    last_x = np.expand_dims(dataset.X[-1], axis=0)
    # Получаем последний год и номер недели
    last_year = dataset.data['Год'].iloc[-1]
    last_week = dataset.data['Номер недели'].iloc[-1]

    for _ in range(n_weeks):
        # Преобразуем last_x в нужную форму и делаем предсказание
        last_x_tensor = torch.tensor(last_x, dtype=torch.float32).view(1, -1)
        with torch.no_grad():
            pred = model(last_x_tensor).detach().numpy().reshape(1, -1)

        # Добавляем предсказание в список
        predictions.append(pred)

        # Обновляем неделю и год
        next_week = last_week + 1
        next_year = last_year
        if next_week > 52:  # Переход на следующий год
            next_week = 1
            next_year += 1
        new_row = np.array([[next_year, next_week] + list(pred[0])])
        last_x = np.append(last_x[:, 1:, :], new_row.reshape(1, 1, -1), axis=1)

        # Обновляем год и неделю для следующей итерации
        last_year = next_year
        last_week = next_week

    return np.array(predictions).reshape(n_weeks, -1)

df = df_dict["Количество исследований по неделям (для реализации).xlsx"].copy()
dataset = CustomDataset(df, window_size=window_size, prev_weeks=prev_weeks)

input_size = window_size * 12  # 10 наблюдений за каждую неделю в окне
hidden_size = 20
output_size = 10  # количество целевых переменных
model = SimpleNN(input_size, hidden_size, output_size)
criterion = CustomHuberLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
model.load_state_dict(torch.load("model/best.pt"))
model.eval()
pred_num_week = 3
result = predict_n_weeks(model, dataset, pred_num_week)
print(result)


[[ 2181.3127   4728.655     669.71173   201.12605 17005.715     930.73175
    577.8397    387.1721  79720.7      6291.27   ]
 [ 2424.427    3227.1257    622.79974   325.45993 17975.773    1538.9012
    996.94946   103.80257 80631.39     8444.229  ]
 [ 1826.709    4062.5608    545.57007    86.00709 14317.014     444.2146
    308.38666   402.66788 69180.73     1779.6624 ]]
